# NVIDIA PhysicsNeMo Sym

## Introductory Example - Lid Driven Cavity Background

https://docs.nvidia.com/deeplearning/physicsnemo/physicsnemo-sym/user_guide/basics/lid_driven_cavity_flow.html

https://github.com/NVIDIA/physicsnemo-sym/blob/2.0.0-rc/examples/ldc/ldc_2d.py[链接文字](https://)

In [1]:
!pip install nvidia-physicsnemo
!pip install nvidia-physicsnemo-sym --no-build-isolation

In [3]:
import os
import warnings

from sympy import Symbol, Eq, Abs

import physicsnemo.sym
from physicsnemo.sym.hydra import to_absolute_path, instantiate_arch, PhysicsNeMoConfig
from physicsnemo.sym.solver import Solver
from physicsnemo.sym.domain import Domain
from physicsnemo.sym.geometry.primitives_2d import Rectangle
from physicsnemo.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
)
from physicsnemo.sym.domain.validator import PointwiseValidator
from physicsnemo.sym.domain.inferencer import PointwiseInferencer
from physicsnemo.sym.key import Key
from physicsnemo.sym.eq.pdes.navier_stokes import NavierStokes
from physicsnemo.sym.utils.io import (
    csv_to_dict,
    ValidatorPlotter,
    InferencerPlotter,
)




In [4]:
!mkdir conf

!mkdir networks

!mkdir openfoam

mkdir: cannot create directory ‘conf’: File exists
mkdir: cannot create directory ‘networks’: File exists
mkdir: cannot create directory ‘openfoam’: File exists


In [5]:
%%writefile conf/config.yaml
defaults :
  - arch:
      - fully_connected
  - scheduler: tf_exponential_lr
  - optimizer: adam
  - loss: sum
  - _self_

scheduler:
  decay_rate: 0.95
  decay_steps: 4000


training:
  rec_validation_freq: 1000
  rec_inference_freq: 2000
  rec_monitor_freq: 1000
  rec_constraint_freq: 2000
  max_steps: 10000
  grad_agg_freq: 1
  save_network_freq: 1000
  print_stats_freq: 100
  summary_freq: 1000
  checkpoint_freq: 1000
  grad_clip_max_norm: 1.0
  monitor_grad_clip: false

  ntk:
    use_ntk: false

  profiler:
    use_profiler: False,
    start_step: 100,
    steps: 10

batch_size:
  TopWall: 1000
  NoSlip: 1000
  Interior: 4000

stop_criterion:
  metric: loss
  mode: min
  min_delta: 1e-5
  patience: 5000
  monitor: loss
  freq: 100
  strict: false

save_filetypes:
  - torch
  - onnx

summary_histograms: false

network_dir: ./networks
initialization_network_dir: ""

run_mode: train
jit: False
amp: default
graph:
  func_arch: true



cuda_graphs:
  enabled: false

cuda_graph_warmup: 10

device: cpu


Overwriting conf/config.yaml


In [7]:

from physicsnemo.sym.hydra.utils import to_yaml
from physicsnemo.sym.hydra.utils import compose
from physicsnemo.sym.hydra.config import PhysicsNeMoConfig

In [8]:
cfg = compose(config_path="conf", config_name="config")

/usr/local/lib/python3.11/dist-packages/physicsnemo/sym/hydra/utils.py:156: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(
/usr/local/lib/python3.11/dist-packages/physicsnemo/sym/hydra/config.py:152: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  logger.warn(


In [9]:
cfg.network_dir = "outputs" # Set the network directory for checkpoints
print(to_yaml(cfg))

arch:
  fully_connected:
    arch_type: fully_connected
    input_keys: ???
    output_keys: ???
    detach_keys: ???
    scaling: null
    layer_size: 512
    nr_layers: 6
    skip_connections: false
    activation_fn: silu
    adaptive_activations: false
    weight_norm: true
scheduler:
  _target_: custom
  _name_: tf.ExponentialLR
  decay_rate: 0.95
  decay_steps: 4000
optimizer:
  _params_:
    compute_gradients: adam_compute_gradients
    apply_gradients: adam_apply_gradients
  _target_: torch.optim.Adam
  lr: 0.001
  betas:
  - 0.9
  - 0.999
  eps: 1.0e-08
  weight_decay: 0.0
  amsgrad: false
loss:
  _target_: physicsnemo.sym.loss.aggregator.Sum
  weights: null
training:
  rec_validation_freq: 1000
  rec_inference_freq: 2000
  rec_monitor_freq: 1000
  rec_constraint_freq: 2000
  max_steps: 10000
  grad_agg_freq: 1
  save_network_freq: 1000
  print_stats_freq: 100
  summary_freq: 1000
  checkpoint_freq: 1000
  grad_clip_max_norm: 1.0
  monitor_grad_clip: false
  ntk:
    use_ntk: 

In [10]:
# make list of nodes to unroll graph on
ns = NavierStokes(nu=0.01, rho=1.0, dim=2, time=False)

# Create a proper architecture configuration
from omegaconf import OmegaConf

# Define a fully connected neural network architecture configuration
arch_config = OmegaConf.create({
    "arch_type": "fully_connected",  # This is the required field
    "layers": [32, 32, 32],          # Hidden layer sizes
    "activation_fn": "silu",         # Activation function
    "skip_connections": False,       # Whether to use skip connections
    "input_normalize": False,        # Whether to normalize inputs
})

# Instantiate the architecture with the proper config
flow_net = instantiate_arch(
    input_keys=[Key("x"), Key("y")],
    output_keys=[Key("u"), Key("v"), Key("p")],
    cfg=arch_config,
)

nodes = ns.make_nodes() + [flow_net.make_node(name="flow_network")]

In [11]:
# add constraints to solver
# make geometry
height = 0.1
width = 0.1
x, y = Symbol("x"), Symbol("y")
rec = Rectangle((-width / 2, -height / 2), (width / 2, height / 2))

# make ldc domain
ldc_domain = Domain()

In [12]:
# top wall
top_wall = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"u": 1.0, "v": 0},
    batch_size=cfg.batch_size.TopWall,
    lambda_weighting={"u": 1.0 - 20 * Abs(x), "v": 1.0},  # weight edges to be zero
    criteria=Eq(y, height / 2),
)
ldc_domain.add_constraint(top_wall, "top_wall")

In [13]:
# no slip
no_slip = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"u": 0, "v": 0},
    batch_size=cfg.batch_size.NoSlip,
    criteria=y < height / 2,
)
ldc_domain.add_constraint(no_slip, "no_slip")

In [15]:
# interior
interior = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"continuity": 0, "momentum_x": 0, "momentum_y": 0},
    batch_size=cfg.batch_size.Interior,
    lambda_weighting={
        "continuity": Symbol("sdf"),
        "momentum_x": Symbol("sdf"),
        "momentum_y": Symbol("sdf"),
    },
)
ldc_domain.add_constraint(interior, "interior")

## NVIDIA Modulus Sym Examples Supplemental Materials

https://xfiles.ngc.nvidia.com/org/nvidia/team/modulus/recipes/modulus_sym_examples_supplemental_materials/versions/latest/files/examples_sym.zip

**openfoam** Google drive share link

https://drive.google.com/drive/folders/1s_LxnqWYEUIPQoEeYabSIBlKJfEmgf2Z?usp=drive_link

In [16]:
# !unzip openfoam.zip

In [17]:
# add validator
file_path = "openfoam/cavity_uniformVel0.csv"
if os.path.exists(to_absolute_path(file_path)):
    mapping = {"Points:0": "x", "Points:1": "y", "U:0": "u", "U:1": "v", "p": "p"}
    openfoam_var = csv_to_dict(to_absolute_path(file_path), mapping)
    openfoam_var["x"] += -width / 2  # center OpenFoam data
    openfoam_var["y"] += -height / 2  # center OpenFoam data
    openfoam_invar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["x", "y"]
    }
    openfoam_outvar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["u", "v"]
    }
    openfoam_validator = PointwiseValidator(
        nodes=nodes,
        invar=openfoam_invar_numpy,
        true_outvar=openfoam_outvar_numpy,
        batch_size=1024,
        plotter=ValidatorPlotter(),
    )
    ldc_domain.add_validator(openfoam_validator)

    # add inferencer data
    grid_inference = PointwiseInferencer(
        nodes=nodes,
        invar=openfoam_invar_numpy,
        output_names=["u", "v", "p"],
        batch_size=1024,
        plotter=InferencerPlotter(),
    )
    ldc_domain.add_inferencer(grid_inference, "inf_data")
else:
    warnings.warn(
        f"Directory {file_path} does not exist. Will skip adding validators. Please download the additional files from NGC https://catalog.ngc.nvidia.com/orgs/nvidia/teams/physicsnemo/resources/Modulus_sym_examples_supplemental_materials"
    )

In [18]:
# make solver
slv = Solver(cfg, ldc_domain)

In [19]:
# Add this before slv.solve()
def debug_metric_dict(metric_dict):
    print("Available metrics:", metric_dict.keys())
    for key, value in metric_dict.items():
        print(f"  {key}: {value.keys() if isinstance(value, dict) else value}")
    return metric_dict

# Monkey patch the _get_score method to debug
from physicsnemo.sym.utils.training.stop_criterion import StopCriterion
original_get_score = StopCriterion._get_score

def patched_get_score(self, metric_dict, target_key):
    debug_metric_dict(metric_dict)
    if target_key not in metric_dict:
        print(f"Warning: '{target_key}' not in metric_dict")
        # Use a default metric if target_key is not available
        available_keys = list(metric_dict.keys())
        if available_keys:
            target_key = available_keys[0]
            print(f"Using '{target_key}' instead")
    return original_get_score(self, metric_dict, target_key)

StopCriterion._get_score = patched_get_score

In [20]:
slv.domain.stop_criterion = None

# start solver
slv.solve()

/usr/local/lib/python3.11/dist-packages/physicsnemo/sym/amp.py:221: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  super().__init__(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at /pytorch/aten/src/ATen/cuda/CublasHandlePool.cpp:180.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/usr/local/lib/python3.11/dist-packages/physicsnemo/sym/eq/derivatives.py:120: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):


Available metrics: dict_keys(['loss', 'validation'])
  loss: dict_keys(['loss', 'u', 'v', 'continuity', 'momentum_y', 'momentum_x'])
  validation: dict_keys(['l2_relative_error_u', 'l2_relative_error_v'])


In [112]:
# !zip -r outputs.zip outputs

  adding: outputs/ (stored 0%)
  adding: outputs/optim_checkpoint.0.pth (deflated 6%)
  adding: outputs/events.out.tfevents.1743097086.fbcfafa56241.2019.9 (deflated 8%)
  adding: outputs/flow_network.0.pth (deflated 8%)
  adding: outputs/events.out.tfevents.1743093984.fbcfafa56241.2019.2 (deflated 8%)
  adding: outputs/events.out.tfevents.1743094448.fbcfafa56241.2019.7 (deflated 8%)
  adding: outputs/events.out.tfevents.1743093659.fbcfafa56241.2019.1 (deflated 50%)
  adding: outputs/validators/ (stored 0%)
  adding: outputs/validators/validator_u.png (deflated 9%)
  adding: outputs/validators/validator_v.png (deflated 8%)
  adding: outputs/constraints/ (stored 0%)
  adding: outputs/constraints/top_wall.vtp (deflated 46%)
  adding: outputs/constraints/interior.vtp (deflated 32%)
  adding: outputs/constraints/no_slip.vtp (deflated 48%)
  adding: outputs/events.out.tfevents.1743094349.fbcfafa56241.2019.5 (deflated 9%)
  adding: outputs/events.out.tfevents.1743094289.fbcfafa56241.2019.4 (d